# Evaluation of WATSEN
## Some housekeeping


In [1]:
# For reloading external modules
%load_ext autoreload
%autoreload 2

In [20]:
import setup
import fetch_videos
import extract_frames
import select_sample_images
import image_datasets
import train_classifier
import json
import os

import evaluation_settings as settings

### Evaluation settings
The method is tested with video data from the floodX experiments.


In [3]:
working_dir='E:/watson_eval'
video_archive_url='https://zenodo.org/record/830451/files/s3_cam1_instar_161007A.tar'
water_level_data_url='https://zenodo.org/record/1014028/files/all_s3_h_us_maxbotix.txt'
camera_time_offset_url='https://zenodo.org/record/830451/files/temporal_offsets_of%20cameras.txt'


### Set up folder structure


In [4]:
stages = setup.run(working_dir)

## Fetch videos

In [5]:
video_folder = fetch_videos.sync(stages[0], video_archive_url)

Download size:  2295258624
s3_cam1_instar_161007A already present - Skipping extraction of E:/watson_eval\1_fetch_videos\downloads\s3_cam1_instar_161007A.tar.


## Extract video frames into multiframe images


In [6]:
# Get water level and time offset data
water_levels = extract_frames.load_water_level(water_level_data_url)
time_offset = extract_frames.load_video_time_offsets(camera_time_offset_url)

In [7]:
image_dirs = []
for timedeltas in settings['frame_extraction_combinations']:
    output_dir = os.path.join(stages[1], '_'.join(str(x) for x in timedeltas))
    extract_frames.extract_from_all(
        video_folder, output_dir, timedeltas, 
        tuple(settings['frame_extraction_new_dim']), 
        water_levels, time_offset, force=False)
    image_dirs.append(output_dir)

## Select samples randomly


In [12]:
image_paths, label_paths = select_sample_images.create(image_dir=image_dirs[3], output_dir=stages[2], settings=settings, image_pattern='*.jpg')

intra-event already exists. use force=True to overwrite
inter-event already exists. use force=True to overwrite


## Create datasets for training


In [18]:
datasets = []
for image_dir in image_dirs:
    datasets.append(image_datasets.create(
        label_dir=label_paths['intra-event'],
        image_dir=image_dir,
        output_dir=stages[3],
        name='cam1_intra_'+os.path.split(image_dir)[1],
        frac_train=0.7, frac_test=0.3))

# Train neural networks


In [21]:
for dataset in datasets:
    train_classifier.train(dataset, stages[4])

2017-10-23 17:25:48,142 Layers 4, features 16, filter size 3x3, pool size: 2x2


Number of files used: 70


2017-10-23 17:25:51,038 Removing 'Q:\Abteilungsprojekte\eng\SWWData\Matthew\Workspace\watsen_unet\scripts\evaluation\prediction'
2017-10-23 17:25:51,040 Removing 'E:\watson_eval\5_train\cam1_intra_0_0_0__ly4ftr16__'
2017-10-23 17:25:51,041 Allocating 'Q:\Abteilungsprojekte\eng\SWWData\Matthew\Workspace\watsen_unet\scripts\evaluation\prediction'
2017-10-23 17:25:51,046 Allocating 'E:\watson_eval\5_train\cam1_intra_0_0_0__ly4ftr16__'


ResourceExhaustedError: OOM when allocating tensor with shape[4,16,356,636]
	 [[Node: Conv2D_1 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Relu, Variable_1/read)]]
	 [[Node: div_1/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_803_div_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D_1', defined at:
  File "c:\opt\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\opt\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\opt\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\opt\python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_